In [41]:
import os
import sys
import shutil
import pandas
import numpy
import ipystata
import matplotlib.pyplot as plt

from ipystata.config import config_stata
config_stata('/home/kmongeon/research/stata14/stata-mp',force_batch=True)

from ipystata.config import config_syntax_higlight    
config_syntax_higlight(True)  

pandas.options.display.max_columns = 100

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pandas.options.display.float_format = '{:.2f}'.format

In [42]:
pwd

'/home/kmongeon/research/exercise_physiology/scripts'

In [43]:
try: 
    shutil.rmtree('out')
    os.mkdir('out')
except:
    pass

In [54]:
%%stata --output dm

import delimited ./data/IzzyLongitudinalWorkingMarch2015.csv,  case(lower) clear 
order id session sequence
sort id sequence
save ./out/di, replace

import delimited ./data/torque.csv, case(lower) clear
drop if session==0
drop sequence
rename new_sequence sequence
order id session sequence
sort id  sequence
foreach var in ptiso pt60 pt240 {
replace `var' = . if `var'==-9999
}
save ./out/dt, replace

** Identify potentially data with errors
* Nota: July  2016
import delimited ./data/NegativeIDs.csv, case(lower) clear
keep nota id sequence 
keep if nota==1
save ./out/db1, replace

* Baraket: Nov 2016
import delimited ./data/data_examine_falk_nov2016.csv, case(lower) clear
keep falk id sequence
keep if falk==1
save ./out/db2, replace

* merge
use ./out/di, clear
sort id sequence
merge 1:1 id sequence using ./out/dt,  nogen
merge 1:1 id sequence using ./out/db1, nogen
merge 1:1 id sequence using ./out/db2, nogen

    
rename radius_sos rsos
rename tibial_sos tsos

rename grip_strength_best grip 
rename mat_offset_new matu

rename godin_pa godin
rename ntxcreat ntxc
rename caloricintake cint
renam calcium calc 
rename vitd vitd 
rename v48 vitd2
rename paq_total_score paq
rename totmvh mvh

rename forearm_length flength
rename thigh_length   tlength

global U id session sequence 
global Y rsos tsos
global X grip ptiso pt60 godin paq ntxc cint calc vitd vitd2 mvh
global Z matu age gender
global D nota falk

drop if matu>5
drop if id==517 & sequence==7

sort id sequence
by id: egen seq = seq()
global U id session sequence seq

order $U $D $Y $X $Z 
sum *
xtset id seq
count 
sort id seq
by id: egen trips = max(seq)
order trips, after(seq)

gen boy = 1 if gender==0
gen girl = 1 if gender==1


replace grip = . if nota==1
*replace rsos = . if nota==1
*replace tsos = . if falk==1
*replace ptiso = . if falk==1



(70 vars, 427 obs)
file ./out/di.dta saved
(14 vars, 421 obs)
(40 observations deleted)(65 real changes made, 65 to missing)
(65 real changes made, 65 to missing)
(65 real changes made, 65 to missing)
file ./out/dt.dta saved
. * Nota: July  2016(32 vars, 109 obs)
(59 observations deleted)
file ./out/db1.dta saved
. import delimited ./data/data_examine_falk_nov2016.csv, case(lower) clear
(34 vars, 473 obs)
(460 observations deleted)
file ./out/db2.dta saved
. use ./out/di, clear

    Result                           # of obs.
    -----------------------------------------
    not matched                            58
        from master                        52  
        from using                          6  

    matched                               375  
    -----------------------------------------

    Result                           # of obs.
    -----------------------------------------
    not matched                           383
        from master                       383

In [55]:
dm['rsoss'] = (dm['rsos'] - dm['rsos'].mean()) / dm['rsos'].std()
dm['tsoss'] = (dm['tsos'] - dm['tsos'].mean()) / dm['tsos'].std()
dm['grips'] = (dm['grip'] - dm['grip'].mean()) / dm['grip'].std()
dm['torqs'] = (dm['ptiso'] - dm['ptiso'].mean()) / dm['ptiso'].std()

dm['ntxs'] = (dm['ntx'] - dm['ntx'].mean()) / dm['ntx'].std()
dm['mvhs'] = (dm['mvh'] - dm['mvh'].mean()) / dm['mvh'].std()
dm['gods'] = (dm['godin'] - dm['godin'].mean()) / dm['godin'].std()
dm['paqs'] = (dm['paq'] - dm['paq'].mean()) / dm['paq'].std()

dm['cints'] = (dm['cint'] - dm['cint'].mean()) / dm['cint'].std()
dm['matus'] = (dm['matu'] - dm['matu'].mean()) / dm['matu'].std()

dm.head(2)

id  session  sequence  seq  trips  nota  falk    rsos    tsos  grip  \
0  100        1         1    1   4.00   nan   nan 3828.00 3601.00   nan   
1  100        1         3    2   4.00   nan   nan 3898.00 3629.00 27.00   

   ptiso  pt60  godin  paq   ntxc    cint    calc  vitd  vitd2    mvh  matu  \
0 113.93 69.29  37.00 2.59 711.85 1572.00 1077.67  3.38 135.27 105.71 -1.67   
1 136.02 87.78  48.00 3.17 760.09 2219.01 1212.82  1.02  41.02  93.62 -0.71   

    age  gender  height  seatedheight  leglength  weight  bf_slaughter  \
0 11.75       0  152.00         75.20      76.80   44.20         22.81   
1 12.71       0  160.80         79.60      81.20   48.80         17.24   

   bia_pbf   bmi  lbmkg  yearsfromaphv  age_at_phv  bone_age  tanner  \
0    21.90 19.13  34.12          -2.13       13.88     11.16    2.00   
1    14.90 18.87  40.39          -1.21       13.92     12.41    4.00   

   menarche age_first_period period_frequency  medications  vitamins  \
0       nan                                           0.00      0.00   
1       nan                                           0.00      0.00   

   fracture  radius_z  tibial_z  wrist_flexion_best  flength  \
0         0      0.80     -0.75                 nan      nan   
1         0      1.00     -0.70               13.65    25.60   

   forearm_lean_csa  forearm_us_csa  anterior_csa  anteriorradius  \
0               nan             nan           nan             nan   
1             30.41           14.49         11.97           34.74   

   ant_rad_csa  anteriorulna  ant_ulna_csa  posterior  post_csa  \
0          nan           nan           nan        nan       nan   
1         9.48         17.81          2.49      17.90      2.52   

   tibialisanterior     ntx  creatinine  caffeine  totsedetary  totlight  \
0               nan 1443.05        2.03     16.10        78.43    174.29   
1               nan 3738.62        4.92      2.14        75.55    148.71   

   totmod  tothard  totvhard  totaltime  paqprvnt  training_data  \
0   94.57    10.14      1.00     804.79      1.00              0   
1   48.79    32.00     12.83     796.04      1.00              1   

   upperarm_length  arm_circumference  arm_lean_csa  tlength  \
0            27.40              24.00         30.77    35.50   
1            30.00              24.20         36.14    36.50   

   thigh_circumference  thigh_lean_csa  bicepsbrachii  v68  quads  v70  \
0                41.70           93.97          26.49 5.51  27.25 5.83   
1                42.30          106.18          23.42 4.31  28.27 6.28   

  biodex_id   visit_one   visit_two         dob  agedays  ageyears  pt240  \
0    B6M100  05-05-2010  05-12-2010  08-11-1998  4292.00     11.75  41.80   
1    B6M200  04-20-2011  04-27-2011  08-11-1998  4642.00     12.71  52.60   

   boy  girl  rsoss  tsoss  grips  torqs  ntxs  mvhs  gods  paqs  cints  matus  
0 1.00   nan   0.12  -0.79    nan  -0.19 -1.10 -0.01 -0.90 -0.84  -0.05  -0.40  
1 1.00   nan   0.82  -0.53   0.47   0.22 -0.55 -0.30 -0.62  0.03   1.29   0.09

In [56]:
%%stata --data dm 
set processors 20

save ./out/dm, replace


    The maximum number of processors or cores being used is changed from 32 tofile ./out/dm.dta saved


In [57]:
dm.nota.value_counts()
dm.falk.value_counts()


1.00    50
Name: nota, dtype: int64

1.00    11
Name: falk, dtype: int64

# Table 1. Summary statatisitcs

In [58]:
dt = dm.groupby('sequence')['rsos', 'tsos', 'grip', 'ptiso', 'ntx', 'matu', 'mvh', 'godin', 'cint', 'paq'].count()


In [59]:
dm[['rsos', 'tsos', 'grip', 'ptiso', 'ntx', 'matu', 'mvh', 'godin', 'cint', 'paq']]

rsos    tsos  grip  ptiso      ntx  matu    mvh  godin    cint  paq
0   3828.00 3601.00   nan 113.93  1443.05 -1.67 105.71  37.00 1572.00 2.59
1   3898.00 3629.00 27.00 136.02  3738.62 -0.71  93.62  48.00 2219.01 3.17
2   3851.00 3677.00   nan 177.05  4027.06  0.41  98.14  44.00 2599.83 3.21
3   3952.00 3740.00 40.50    nan  2631.34  1.17  88.14  66.00 2482.12 2.76
4   3682.00 3603.00   nan 133.05  9869.88 -1.63  89.29  25.00 1549.99 1.71
5   3935.00 3916.00   nan 171.36  8867.85 -1.24  86.57  45.00 1308.95 2.95
6   3899.00 3863.00 27.00    nan 10374.04 -0.27  68.71 102.00 1260.66 3.83
7   3878.00 3908.00 42.00 211.00 18145.79  0.77  59.14 113.00 2509.72 3.36
8   3951.00 4015.00 42.00    nan      nan  2.08  60.71 123.00 1685.95 2.47
9   3768.00 3686.00   nan 146.60  6223.78 -1.91 120.57  33.00 2551.88 3.26
10  3790.00 3750.00 22.00 192.37 10328.49 -0.96  82.00  76.00 1822.43 2.66
11  3886.00 3882.00 32.50    nan  2208.56  1.11    nan  85.00 2452.76 3.10
12  3635.00 3536.00   nan 170.15  8247.66 -0.19  67.00  59.00 3624.72 2.24
13  3692.00 3578.00 32.00 186.81      nan  0.70    nan   0.00 1499.16 1.25
14  3859.00 3619.00   nan    nan      nan -2.83 123.86 174.00 1612.47 4.03
15  3924.00 3627.00   nan 114.00  1793.72 -1.94 128.68  80.00 1315.71 3.83
16  3889.00 3654.00   nan 164.85  5698.50 -0.32    nan  56.00 1314.36 3.24
17  3829.00 3601.00   nan  84.77  2600.33 -2.54    nan  40.00 1652.88 3.30
18  3812.00 3651.00   nan    nan  7895.91 -1.59  58.43  29.00 1242.90 2.85
19  3749.00 3662.00   nan 140.67  6725.25 -0.53  69.00  78.00 1918.74 2.55
20  3821.00 3858.00   nan  91.99  5154.53 -3.56    nan  59.00  901.72 3.96
21  3935.00 3759.00   nan  98.43  3063.60 -2.62 134.29  73.00 1704.68 3.96
22  3888.00 3764.00   nan 149.60  5340.75 -1.50    nan  72.00 2351.09 4.03
23  3895.00 3768.00 27.50    nan      nan -0.76  87.33  75.00 2105.88 4.18
24  3776.00 3601.00   nan  91.31  3991.68 -2.99 155.96  70.00 1322.41 3.52
25  3770.00 3660.00 18.50 105.65 10315.60 -2.05  96.29  75.00 1685.67 3.47
26  4019.00 3764.00   nan 177.09  5404.34 -1.27 141.71  82.00 1804.18 4.10
27  3850.00 3689.00 33.50 248.62 10074.59 -0.41 163.05  63.00 2879.12 3.04
28  3886.00 3717.00 40.00 314.51 15981.35  0.66  89.71 210.00 2687.38 3.50
29  3927.00 3842.00 49.00    nan  5139.97  1.86 102.29 122.00 2234.86 3.84
..      ...     ...   ...    ...      ...   ...    ...    ...     ...  ...
395 3967.00 3701.00 19.50    nan  5817.13 -1.76    nan  88.00 1659.15 3.48
396 3873.00 3910.00 32.50    nan   980.31  3.27  13.57  13.00 1318.95 1.44
397 3901.00 3672.00 27.50 134.76  2423.86  2.53  86.86  56.00 1184.99 2.56
398 3930.00 3868.00 30.00 138.51  2609.01  3.53  61.14  17.00 1951.69 1.63
399 3771.00 3850.00 14.75    nan 13058.56 -0.74  69.79  68.00 2142.94 3.58
400 3864.00 3943.00 18.00    nan  6005.58  0.25    nan  68.00 1034.26 2.83
401 3597.00 3645.00 16.00  84.15  6627.98 -0.28 106.86  59.00 1659.74 2.87
402 3794.00 3715.00 25.00 126.54  7684.01  0.76    nan  60.00  948.22 2.25
403 3905.00 3771.00 24.50 130.63  2722.69  1.74 106.86  56.00 1005.17 3.18
404 4098.00 3842.00 28.00 214.50   913.01  4.05  37.89  55.00 2016.67 1.81
405 4140.00 3931.00 31.50 194.82   899.65  4.95  24.86   8.00 1886.54 1.42
406 3832.00 3638.00 25.00 165.26  6798.33  0.90  52.86  43.00 1476.62 2.65
407 3686.00 3715.00 17.50 121.46  3754.85 -1.31  94.86  14.00 1335.53 2.38
408 3806.00 3595.00 15.50  60.13  7381.56 -2.47 153.71  91.00 1899.62 3.48
409 3896.00 3761.00 18.50  65.82  6145.81 -1.52 136.86 108.00 1399.25 3.86
410 3814.00 3567.00 17.25 107.66  6441.02 -0.62  53.86 132.00 1410.50 3.10
411 3920.00 3712.00 18.50  93.10 21921.06  0.29  78.14  49.00 1450.11 2.20
412 3957.00 3644.00   nan  82.99  3994.00 -2.00  94.00 127.00 1025.71 4.04
413 3880.00 3672.00   nan 104.53  4696.85 -1.06 112.86 101.00 1024.41 3.81
414 3820.00 3474.00 15.00  79.28  3966.80 -2.68  67.71 119.00 1463.98 3.60
415 3832.00 3529.00 16.50  77.80  1879.55 -1.77 118.14 121.00 1568.11 3.71
416 3853.00 3694.00 23.00    nan

In [60]:
ds = dm.groupby('seq')['rsos', 'tsos', 'grip', 'ptiso'].mean()
dm.groupby('seq')['rsos', 'tsos', 'grip', 'ptiso', 'ntx'].count()

d0 = dm[dm['gender']==0].groupby('seq')['rsos', 'tsos', 'grip', 'ptiso'].mean()
d1 = dm[dm['gender']==1].groupby('seq')['rsos', 'tsos', 'grip', 'ptiso'].mean()

t1 = ds.append(d0)
t1 = t1.append(d1)
#t1.to_csv("./out/t1.csv")
t1

rsos  tsos  grip  ptiso  ntx
seq                              
1     173   174   113    140  170
2     139   139   122    110  130
3      86    85    78     60   82
4      17    17    16      0    4

rsos    tsos  grip  ptiso
seq                             
1   3792.21 3654.82 21.14 111.62
2   3820.22 3687.64 22.87 127.48
3   3842.10 3729.59 26.21 146.71
4   3880.53 3800.12 30.97    nan
1   3791.63 3652.71 21.32 116.70
2   3809.36 3676.25 24.29 131.64
3   3822.49 3705.89 28.82 157.85
4   3842.88 3788.75 36.29    nan
1   3792.89 3657.30 20.94 106.24
2   3831.90 3699.88 21.40 123.47
3   3863.63 3756.25 23.73 133.10
4   3914.00 3810.22 26.83    nan

In [61]:
ds.iloc[3]/ds.iloc[0]-1
d0.iloc[3]/d0.iloc[0]-1
d1.iloc[3]/d1.iloc[0]-1

rsos    0.02
tsos    0.04
grip    0.47
ptiso    nan
dtype: float64

rsos    0.01
tsos    0.04
grip    0.70
ptiso    nan
dtype: float64

rsos    0.03
tsos    0.04
grip    0.28
ptiso    nan
dtype: float64

In [62]:
%%stata --data dm 
set processors 20
keep id sequence rsos tsos grip ptiso ntx matu mvh godin cint paq
xtset id sequence

xtsum *


    The maximum number of processors or cores being used is changed from 32 to       panel variable:  id (unbalanced)
        time variable:  sequence, 1 to 7, but with gaps
                delta:  1 unit

Variable         |      Mean   Std. Dev.       Min        Max |    Observations
-----------------+--------------------------------------------+----------------
id       overall |  326.1976   197.5498        100        583 |     N =     425
         between |             197.2537        100        583 |     n =     178
         within  |                    0   326.1976   326.1976 | T-bar = 2.38764
                 |                                            |
sequence overall |  4.197647   1.833518          1          7 |     N =     425
         between |             1.078822          1          6 |     n =     178
         within  |              1.59697   1.197647    7.53098 | T-bar = 2.38764
                 |                                            |
rsos     overall |  3815.

# Figure 2. Impact of muscle strength on bone properties

In [16]:
%%stata --data dm 
set processors 20



xtset id seq
gsem ///
    (M1[id] grips matus ntxs paqs  -> rsoss, ) /// 
    (M2[id] torqs matus ntxs mvhs  -> tsoss, ) /// 
    (cints matus  -> grips,)  ///
    (cints matus  -> torqs,)  ///
     , covstruct(_lexogenous, diagonal) nocapslatent ///
      latent(M1 M2)   ///
      cov(e.rsoss*e.tsoss)  cov(e.torqs*e.grips)  ///
      cov(e.rsoss*e.grips)  cov(e.tsoss*e.torqs) ///
      cov(M1[id]*M2[id])  
        
estimates save "./out/table2_b", replace

lincom [grips]matus - [torqs]matus
lincom [grips]cints - [torqs]cints

lincom [rsoss]grips - [tsoss]torqs
lincom [rsoss]matus - [tsoss]matus
lincom [rsoss]ntxs - [tsoss]ntxs
lincom [rsoss]paqs - [tsoss]mvhs





    The maximum number of processors or cores being used is changed from 32 to(50 real changes made, 50 to missing)
. replace tsos = . if falk==1
(11 real changes made, 11 to missing)
(11 real changes made, 11 to missing)
       panel variable:  id (unbalanced)
        time variable:  seq, 1 to 4
                delta:  1 unit
>     (M1[id] grips matus ntxs paqs  -> rsoss, ) /// 
>     (M2[id] torqs matus ntxs mvhs  -> tsoss, ) /// 
>     (cints matus  -> grips,)  ///
>     (cints matus  -> torqs,)  ///
>      , covstruct(_lexogenous, diagonal) nocapslatent ///
>       latent(M1 M2)   ///
>       cov(e.rsoss*e.tsoss)  cov(e.torqs*e.grips)  ///
>       cov(e.rsoss*e.grips)  cov(e.tsoss*e.torqs) ///
>       cov(M1[id]*M2[id])  

Fitting fixed-effects model:

Iteration 0:   log likelihood = -1466.4789  (not concave)
Iteration 1:   log likelihood = -1410.1501  
Iteration 2:   log likelihood = -1383.7297  
Iteration 3:   log likelihood = -1382.4273  
Iteration 4:   log likelihood = -1382.3

# Figure 3. Constrained specification results.

In [ ]:
%%stata --data dm 
set processors 20

estimates use "./out/table2_b"
matrix b = e(b)
matrix list b

***********************************************************************

constraint 1 _b[rsoss:grips] = _b[tsoss:torqs]
constraint 2 _b[rsoss:matus]  = _b[tsoss:matus]
constraint 3 _b[rsoss:ntxs]  = _b[tsoss:ntxs]
constraint 4 _b[rsoss:paqs]  = _b[tsoss:mvhs]

constraint 5 _b[grips:cints] = _b[torqs:cints]
constraint 6 _b[grips:matus]  = _b[torqs:matus]


constraint 10 _b[grips:_cons] = 0 
constraint 11 _b[torqs:_cons] = 0 
constraint 12 _b[rsoss:_cons] = 0 
constraint 13 _b[tsoss:_cons] = 0 

constraint 14 [var(e.rsoss)]_cons - [var(e.tsoss)]_cons = 0
constraint 15 [var(e.grips)]_cons - [var(e.torqs)]_cons = 0
        
gsem ///
    (M1[id]@1 grips matus ntxs paqs  -> rsoss, ) /// 
    (M1[id]@1 torqs matus ntxs mvhs  -> tsoss, ) /// 
    (cints matus  -> grips,)  ///
    (cints matus  -> torqs,)  ///
     , covstruct(_lexogenous, diagonal) nocapslatent ///
      latent(M1)   ///
      cov(e.rsoss*e.tsoss)  cov(e.torqs*e.grips)  ///
      cov(e.rsoss*e.grips)  cov(e.tsoss*e.torqs) ///
      constraints(1 2 3 4 5 6  10 11 12 13 14 15) ///
      from(b, skip)

** indirect effect      
nlcom _b[rsoss:c.grips]*_b[grips:c.matus]
** total effect      
nlcom _b[rsoss:c.grips] + _b[rsoshs:c.grips]*_b[grips:c.matus]

# Table 2. Sex-specific estimation results.

In [80]:
%%stata --data dm 
set processors 20

label define sex 0 "boys" 1 "girls
label values gender sex  



constraint 10 _b[griphs:_cons] = 0 
constraint 11 _b[torqhs:_cons] = 0 
constraint 12 _b[rsoshs:_cons] = 0 
constraint 13 _b[tsoshs:_cons] = 0 


constraint 20 [var(e.rsoshs)]_cons - [var(e.tsoshs)]_cons = 0
constraint 21 [var(e.griphs)]_cons - [var(e.torqhs)]_cons = 0
/*
gsem ///
    (M1[id]@1 i1.gender#c.griphs i1.gender#c.matus i1.gender#c.ntxhs i1.gender#c.paqps  -> rsoshs, ) /// 
    (M1[id]@1 i1.gender#c.torqhs i1.gender#c.matus i1.gender#c.ntxhs i1.gender#c.mvhhs  -> tsoshs, ) /// 
    ///
    (M1[id]@1 i0.gender#c.griphs i0.gender#c.matus i0.gender#c.ntxhs i1.gender#c.paqps  -> rsoshs, ) /// 
    (M1[id]@1 i0.gender#c.torqhs i0.gender#c.matus i0.gender#c.ntxhs i0.gender#c.mvhhs  -> tsoshs, ) /// 
    ///
    (i1.gender#c.godhs i1.gender#c.cinths i1.gender#c.matus    -> griphs,)  ///
    (i1.gender#c.godhs i1.gender#c.cinths i1.gender#c.matus    -> torqhs,)  ///
    ///
    (i0.gender#c.godhs i0.gender#c.cinths i0.gender#c.matus  -> griphs,)  ///
    (i0.gender#c.godhs i0.gender#c.cinths i0.gender#c.matus  -> torqhs,)  ///
    , covstruct(_lexogenous, diagonal) nocapslatent ///
    latent(M1) constraints(10 11 12 13 20 21) ///
    cov(e.rsoshs*e.tsoshs)  cov(e.torqhs*e.griphs)  ///
    cov(e.rsoshs*e.griphs)  cov(e.tsoshs*e.torqhs)
     
estimates store t3a
estimates save "./out/t3", replace

lincom _b[rsoshs:0b.gender#c.griphs] - _b[tsoshs:0b.gender#c.torqhs]
lincom _b[rsoshs:i1.gender#c.griphs] - _b[tsoshs:i1.gender#c.torqhs]
lincom _b[rsoshs:0b.gender#c.matus] - _b[tsoshs:0b.gender#c.matus]
lincom _b[rsoshs:1.gender#c.matus] - _b[tsoshs:1.gender#c.matus]
lincom _b[rsoshs:0b.gender#c.ntxhs] - _b[tsoshs:0b.gender#c.ntxhs]
lincom _b[rsoshs:1.gender#c.ntxhs] - _b[tsoshs:1.gender#c.ntxhs]   
lincom _b[griphs:0b.gender#c.matus] - _b[torqhs:0b.gender#c.matus]    
lincom _b[griphs:1.gender#c.matus] - _b[torqhs:1.gender#c.matus]
*/
***********************************************************************
***********************************************************************
gsem ///
    (M1[id]@1 i1.gender#c.griphs@b1a i1.gender#c.matus@b1b i1.gender#c.ntxhs@b1c i1.gender#c.paqps@b1d  -> rsoshs, ) /// 
    (M1[id]@1 i1.gender#c.torqhs@b1a i1.gender#c.matus@b1b i1.gender#c.ntxhs@b1c i1.gender#c.mvhhs@b1d  -> tsoshs, ) /// 
    ///
    (M1[id]@1 i0.gender#c.griphs@b0a i0.gender#c.matus@b0b i0.gender#c.ntxhs@b0c i0.gender#c.paqps@b0d  -> rsoshs, ) /// 
    (M1[id]@1 i0.gender#c.torqhs@b0a i0.gender#c.matus@b0b i0.gender#c.ntxhs@b0c i0.gender#c.mvhhs@b0d  -> tsoshs, ) /// 
    ///
    (i1.gender#c.godhs@m1a i1.gender#c.cinths@m1b i1.gender#c.matus@m1c    -> griphs,)  ///
    (i1.gender#c.godhs@m1a i1.gender#c.cinths@m1b i1.gender#c.matus@m1c    -> torqhs,)  ///
    ///
    (i0.gender#c.godhs@m0a i0.gender#c.cinths@m0b i0.gender#c.matus@m0c  -> griphs,)  ///
    (i0.gender#c.godhs@m0a i0.gender#c.cinths@m0b i0.gender#c.matus@m0c  -> torqhs,)  ///
    , covstruct(_lexogenous, diagonal) nocapslatent ///
    latent(M1) constraints(10 11 12 13 20 21) ///
    cov(e.rsoshs*e.tsoshs)  cov(e.torqhs*e.griphs)  ///
    cov(e.rsoshs*e.griphs)  cov(e.tsoshs*e.torqhs)

     estimates store t3b
estimates save "./out/t3b", replace

***********************************************************************
***muscle strength

lincom _b[griphs:1.gender#c.matus] - _b[griphs:0.gender#c.matus]

***************************
***nutrition
  
lincom _b[griphs:1.gender#c.cinths] - _b[griphs:0.gender#c.cinths]       
          
***************************
  
lincom _b[griphs:1.gender#c.godhs] - _b[griphs:0.gender#c.godhs] 
***************************
***bone properties
         
lincom _b[rsoshs:1.gender#c.matus] - _b[rsoshs:0.gender#c.matus]
***************************
    
lincom _b[rsoshs:1.gender#c.ntxhs] - _b[rsoshs:0.gender#c.ntxhs]
***************************
 
lincom _b[rsoshs:1.gender#c.griphs] - _b[rsoshs:0.gender#c.griphs]
          
***************************

lincom _b[tsoshs:1.gender#c.mvhhs] - _b[tsoshs:0.gender#c.mvhhs]          
          
***********************************************************************
****decompose effects maturation effect
          
****boys
**direct effect
 
*matu on bone          
nlcom _b[rsoshs:0b.gender#c.matus]    
         
** indirect effect 
*grip on bone
nlcom _b[rsoshs:0b.gender#c.griphs]

*matu on grip
nlcom _b[griphs:0b.gender#c.matus]

*product (indirect effect)
nlcom _b[rsoshs:0b.gender#c.griphs]*_b[griphs:0b.gender#c.matus]
         
** total effect      
    
nlcom _b[rsoshs:0b.gender#c.matus] 
nlcom _b[rsoshs:0b.gender#c.griphs]*_b[griphs:0b.gender#c.matus]
nlcom _b[rsoshs:0b.gender#c.matus] + _b[rsoshs:0b.gender#c.griphs]*_b[griphs:0b.gender#c.matus]

***girls
**direct effect
 
*matu on bone          
nlcom _b[rsoshs:i1.gender#c.matus]    
         
** indirect effect 
*grip on bone
nlcom _b[rsoshs:i1.gender#c.griphs]

*matu on grip
nlcom _b[griphs:i1.gender#c.matus]

*product (indirect effect)
nlcom _b[rsoshs:i1.gender#c.griphs]*_b[griphs:i1.gender#c.matus]
         
** total effect      
    
nlcom _b[rsoshs:i1.gender#c.matus] 
nlcom _b[rsoshs:i1.gender#c.griphs]*_b[griphs:i1.gender#c.matus]
nlcom _b[rsoshs:i1.gender#c.matus] + _b[rsoshs:i1.gender#c.griphs]*_b[griphs:i1.gender#c.matus]
         
************************************************************************************
************************************************************************************
************************************************************************************
 
*estout t3a t3b , cells("b( fmt(2)) ci(par fmt(2))") interaction(:) label 
         


    The maximum number of processors or cores being used is changed from 32 to
. > gsem ///
>     (M1[id]@1 i1.gender#c.griphs i1.gender#c.matus i1.gender#c.ntxhs i1.gender#c.paqps  -> rsoshs, ) /// 
>     (M1[id]@1 i1.gender#c.torqhs i1.gender#c.matus i1.gender#c.ntxhs i1.gender#c.mvhhs  -> tsoshs, ) /// 
>     ///
>     (M1[id]@1 i0.gender#c.griphs i0.gender#c.matus i0.gender#c.ntxhs i1.gender#c.paqps  -> rsoshs, ) /// 
>     (M1[id]@1 i0.gender#c.torqhs i0.gender#c.matus i0.gender#c.ntxhs i0.gender#c.mvhhs  -> tsoshs, ) /// 
>     ///
>     (i1.gender#c.godhs i1.gender#c.cinths i1.gender#c.matus    -> griphs,)  ///
>     (i1.gender#c.godhs i1.gender#c.cinths i1.gender#c.matus    -> torqhs,)  ///
>     ///
>     (i0.gender#c.godhs i0.gender#c.cinths i0.gender#c.matus  -> griphs,)  ///
>     (i0.gender#c.godhs i0.gender#c.cinths i0.gender#c.matus  -> torqhs,)  ///
>     , covstruct(_lexogenous, diagonal) nocapslatent ///
>     latent(M1) constraints(10 11 12 13 20 21) ///
>     cov(e

# Table 3a. Male-specific results.

In [50]:
%%stata --data dm 
set processors 20

constraint 1 _b[rsoshs:griphs] = _b[tsoshs:torqhs]
constraint 2 _b[rsoshs:matus]  = _b[tsoshs:matus]
constraint 3 _b[rsoshs:ntxhs]  = _b[tsoshs:ntxhs]
constraint 4 _b[rsoshs:paqps]  = _b[tsoshs:mvhhs]

constraint 5 _b[griphs:godhs]  = _b[torqhs:godhs]
constraint 6 _b[griphs:cinths] = _b[torqhs:cinths]
constraint 7 _b[griphs:matus]  = _b[torqhs:matus]


constraint 10 _b[griphs:_cons] = 0 
constraint 11 _b[torqhs:_cons] = 0 
constraint 12 _b[rsoshs:_cons] = 0 
constraint 13 _b[tsoshs:_cons] = 0 

constraint 14 [var(e.rsoshs)]_cons - [var(e.tsoshs)]_cons = 0
constraint 15 [var(e.griphs)]_cons - [var(e.torqhs)]_cons = 0

gsem ///
    (M1[id] griphs matus ntxhs paqps  -> rsoshs, ) /// 
    (M2[id] torqhs matus ntxhs mvhhs  -> tsoshs, ) /// 
    (godhs cinths matus    -> griphs,)  ///
    (godhs cinths matus    -> torqhs,)  ///
     , covstruct(_lexogenous, diagonal) nocapslatent ///
      latent(M1 M2)   ///
      cov(e.rsoshs*e.tsoshs)  cov(e.torqhs*e.griphs)  ///
      cov(e.rsoshs*e.griphs)  cov(e.tsoshs*e.torqhs) ///
      cov(M1[id]*M2[id])  
matrix b = e(b)

gsem ///
    (M1[id]@1 griphs matus ntxhs paqps  -> rsoshs, ) /// 
    (M1[id]@1 torqhs matus ntxhs mvhhs  -> tsoshs, ) /// 
    ( godhs cinths matus    -> griphs,)  ///
    ( godhs cinths matus    -> torqhs,)  ///
     if gender==0, covstruct(_lexogenous, diagonal) nocapslatent ///
      latent(M1 M2)   ///
      cov(e.rsoshs*e.tsoshs)  cov(e.torqhs*e.griphs)  ///
      cov(e.rsoshs*e.griphs)  cov(e.tsoshs*e.torqhs) ///
      constraints(1 2 3 4 5 6 7  10 11 12 13 14 15) ///
      from(b, skip)





    The maximum number of processors or cores being used is changed from 32 to>     (M1[id] griphs matus ntxhs paqps  -> rsoshs, ) /// 
>     (M2[id] torqhs matus ntxhs mvhhs  -> tsoshs, ) /// 
>     (godhs cinths matus    -> griphs,)  ///
>     (godhs cinths matus    -> torqhs,)  ///
>      , covstruct(_lexogenous, diagonal) nocapslatent ///
>       latent(M1 M2)   ///
>       cov(e.rsoshs*e.tsoshs)  cov(e.torqhs*e.griphs)  ///
>       cov(e.rsoshs*e.griphs)  cov(e.tsoshs*e.torqhs) ///
>       cov(M1[id]*M2[id])  

Fitting fixed-effects model:

Iteration 0:   log likelihood = -2050.3787  (not concave)
Iteration 1:   log likelihood =  -1986.265  
Iteration 2:   log likelihood = -1982.0747  
Iteration 3:   log likelihood = -1971.8136  
Iteration 4:   log likelihood = -1970.9114  
Iteration 5:   log likelihood =  -1970.826  
Iteration 6:   log likelihood = -1970.8253  
Iteration 7:   log likelihood = -1970.8253  

Refining starting values:

Grid node 0:   log likelihood = -1947.9747

Fi

# Table 3b. Girl-specific results.

In [48]:
%%stata --data dm 
set processors 20

constraint 1 _b[rsoshs:griphs] = _b[tsoshs:torqhs]
constraint 2 _b[rsoshs:matus]  = _b[tsoshs:matus]
constraint 3 _b[rsoshs:ntxhs]  = _b[tsoshs:ntxhs]
constraint 4 _b[rsoshs:paqps]  = _b[tsoshs:mvhhs]

constraint 5 _b[griphs:godhs]  = _b[torqhs:godhs]
constraint 6 _b[griphs:cinths] = _b[torqhs:cinths]
constraint 7 _b[griphs:matus]  = _b[torqhs:matus]

constraint 10 _b[griphs:_cons] = 0 
constraint 11 _b[torqhs:_cons] = 0 
constraint 12 _b[rsoshs:_cons] = 0 
constraint 13 _b[tsoshs:_cons] = 0 

constraint 14 [var(e.rsoshs)]_cons - [var(e.tsoshs)]_cons = 0
constraint 15 [var(e.griphs)]_cons - [var(e.torqhs)]_cons = 0

gsem ///
    (M1[id] griphs matus ntxhs paqps  -> rsoshs, ) /// 
    (M2[id] torqhs matus ntxhs mvhhs  -> tsoshs, ) /// 
    (godhs cinths matus    -> griphs,)  ///
    (godhs cinths matus    -> torqhs,)  ///
     , covstruct(_lexogenous, diagonal) nocapslatent ///
      latent(M1 M2)   ///
      cov(e.rsoshs*e.tsoshs)  cov(e.torqhs*e.griphs)  ///
      cov(e.rsoshs*e.griphs)  cov(e.tsoshs*e.torqhs) ///
      cov(M1[id]*M2[id])  
matrix b = e(b)

gsem ///
    (M1[id]@1 griphs matus ntxhs paqps  -> rsoshs, ) /// 
    (M1[id]@1 torqhs matus ntxhs mvhhs  -> tsoshs, ) /// 
    ( godhs cinths matus    -> griphs,)  ///
    ( godhs cinths matus    -> torqhs,)  ///
     if gender==1, covstruct(_lexogenous, diagonal) nocapslatent ///
      latent(M1 M2)   ///
      cov(e.rsoshs*e.tsoshs)  cov(e.torqhs*e.griphs)  ///
      cov(e.rsoshs*e.griphs)  cov(e.tsoshs*e.torqhs) ///
      constraints(1 2 3 4 5 6 7  10 11 12 13 14 15) ///
      from(b, skip)

** indirect effect      
nlcom _b[rsoshs:c.griphs]*_b[griphs:c.matus]
** total effect      
nlcom _b[rsoshs:c.griphs] + _b[rsoshs:c.griphs]*_b[griphs:c.matus]


    The maximum number of processors or cores being used is changed from 32 to>     (M1[id] griphs matus ntxhs paqps  -> rsoshs, ) /// 
>     (M2[id] torqhs matus ntxhs mvhhs  -> tsoshs, ) /// 
>     (godhs cinths matus    -> griphs,)  ///
>     (godhs cinths matus    -> torqhs,)  ///
>      , covstruct(_lexogenous, diagonal) nocapslatent ///
>       latent(M1 M2)   ///
>       cov(e.rsoshs*e.tsoshs)  cov(e.torqhs*e.griphs)  ///
>       cov(e.rsoshs*e.griphs)  cov(e.tsoshs*e.torqhs) ///
>       cov(M1[id]*M2[id])  

Fitting fixed-effects model:

Iteration 0:   log likelihood = -2050.3787  (not concave)
Iteration 1:   log likelihood =  -1986.265  
Iteration 2:   log likelihood = -1982.0747  
Iteration 3:   log likelihood = -1971.8136  
Iteration 4:   log likelihood = -1970.9114  
Iteration 5:   log likelihood =  -1970.826  
Iteration 6:   log likelihood = -1970.8253  
Iteration 7:   log likelihood = -1970.8253  

Refining starting values:

Grid node 0:   log likelihood = -1947.9747

Fi